# Ćwiczenia praktyczne – Łączenie i analiza danych w pandas

Ten zeszyt zawiera **samodzielne zadania** inspirowane pracą z wieloma arkuszami w Excelu, ale wykonywane w bibliotece **pandas**.

> **Jak pracować?**
> 1. Uruchom kolejno komórki z kodem.
> 2. W miejscach oznaczonych `# TODO` uzupełnij brakujące linie.
> 3. Skorzystaj z podpowiedzi w sekcji **Wskazówka**.
> 4. Sprawdź wynik przy pomocy `head()`, `shape`, `describe()`.

---

## 1. Przygotowanie środowiska i przykładowych danych

In [ ]:
import pandas as pd
import numpy as np

np.random.seed(42)

# Tabela Customers
customers_df = pd.DataFrame({
    'customer_id': range(1, 11),
    'customer_name': ['Klient_'+str(i) for i in range(1, 11)],
    'country': np.random.choice(['Polska', 'Niemcy', 'Francja', 'Hiszpania'], size=10)
})

# Tabela Products
products_df = pd.DataFrame({
    'product_id': range(101, 121),
    'product_name': [f'Produkt_{i}' for i in range(101, 121)],
    'category': np.random.choice(['Electronics', 'Home', 'Toys', 'Books'], size=20),
    'unit_price': np.random.uniform(5, 200, size=20).round(2)
})

# Tabela Orders
dates = pd.date_range('2024-01-01', '2024-12-31', freq='D')
orders_df = pd.DataFrame({
    'order_id': range(1001, 1101),
    'customer_id': np.random.choice(customers_df['customer_id'], size=100),
    'order_date': np.random.choice(dates, size=100),
    'product_id': np.random.choice(products_df['product_id'], size=100),
    'quantity': np.random.randint(1, 5, size=100)
})

# Tabela Returns (losowy podzbiór zamówień)
returned_orders = np.random.choice(orders_df['order_id'], size=15, replace=False)
returns_df = pd.DataFrame({
    'order_id': returned_orders,
    'return_date': pd.to_datetime(np.random.choice(pd.date_range('2024-02-01', '2025-01-31'), size=15)),
    'reason': np.random.choice(['Damaged', 'Wrong Item', 'No Longer Needed'], size=15)
})

# Podgląd danych
for name, df in [('Customers', customers_df),
                 ('Products', products_df.head()),
                 ('Orders', orders_df.head()),
                 ('Returns', returns_df.head())]:
    print(f"--- {name} ---")
    display(df.head())

## 2. Ćwiczenie 1 – Łączenie zamówień z produktami
Połącz **orders_df** z **products_df** tak, aby uzyskać pełną tabelę z nazwą produktu, kategorią i ceną jednostkową. Następnie oblicz nową kolumnę `line_total`, czyli *quantity × unit_price*.

**Wskazówka:** Użyj `pd.merge()` na kolumnie `product_id`, a potem prostej operacji arytmetycznej.

In [ ]:
# TODO: scal orders_df z products_df
orders_products_df = (
    orders_df
    # .merge(...)
)

# TODO: dodaj kolumnę line_total
# orders_products_df['line_total'] = ...

orders_products_df.head()

## 3. Ćwiczenie 2 – Przychód per kraj
Dołącz dane klientów, aby każdemu zamówieniu przypisać kraj. Następnie oblicz **łączny przychód** (suma `line_total`) dla każdego kraju.

**Wskazówka:**
1. Najpierw scal wynik z ćw. 1 z **customers_df** (`customer_id`).
2. Następnie skorzystaj z `groupby('country')['line_total'].sum()`.
3. Posortuj malejąco.

In [ ]:
# TODO: wynik z ćw. 1 + customers_df
orders_full_df = (
    orders_products_df
    # .merge(...)
)

# TODO: agregacja przychodu per kraj
# revenue_by_country = ...

orders_full_df.head()
# revenue_by_country

## 4. Ćwiczenie 3 – Tabela przestawna
Stwórz **pivot table**, w której wierszami będą **miesiące** (na podstawie `order_date`), kolumnami **kategorie produktów**, a wartościami **łączna sprzedaż** (`line_total`). Dodaj marginesy (kolumnę i wiersz „Total”).

**Wskazówka:**
* Wydobądź miesiąc: `orders_full_df['order_month'] = orders_full_df['order_date'].dt.to_period('M')`.
* Użyj `pd.pivot_table()` z `aggfunc='sum'`, `margins=True`.

In [ ]:
# TODO: dodaj kolumnę order_month
# orders_full_df['order_month'] = ...

# TODO: pivot table
# sales_pivot = pd.pivot_table(...)

# sales_pivot.head()

## 5. Ćwiczenie 4 – Top 3 klientów
Znajdź trzech klientów, którzy wygenerowali **najwyższy łączny przychód** w 2024 roku.

**Wskazówka:**
* Skorzystaj z `groupby('customer_name')['line_total'].sum()` i `nlargest(3)`.

In [ ]:
# TODO: oblicz przychód per klient
# top_customers = ...

# top_customers

## 6. Ćwiczenie 5 – Wskaźnik zwrotów
Policz **procent** zamówień, które zostały zwrócone, zarówno w skali całej firmy, jak i dla każdej kategorii produktu.

**Wskazówka:**
1. Ustal zestaw `order_id` zwróconych zamówień (`returns_df['order_id']`).
2. Utwórz kolumnę `is_returned` w tabeli z ćw. 1 (`orders_products_df['order_id'].isin(...)`).
3. a) Całkowity % zwrotów: średnia kolumny `is_returned` × 100.
   b) % zwrotów per kategoria: `groupby('category')['is_returned'].mean()*100`.

In [ ]:
# TODO: dodaj kolumnę is_returned
# orders_products_df['is_returned'] = ...

# TODO: procent zwrotów globalnie
# return_rate_all = ...

# TODO: procent zwrotów per kategoria
# return_rate_by_cat = ...

# return_rate_all, return_rate_by_cat

## 7. Ćwiczenie 6 – Eksport wyników
Zapisz pivot table z ćw. 3 do pliku **CSV** i **Excel**:
* plik `sales_pivot.csv` (bez indeksu),
* plik `sales_pivot.xlsx` (arkusz "SalesByMonth").

**Wskazówka:**
* `to_csv('ścieżka', index=False)`
* `to_excel('ścieżka', sheet_name='SalesByMonth')`.

In [ ]:
# TODO: zapisz pliki
# csv_path = '/mnt/data/sales_pivot.csv'
# excel_path = '/mnt/data/sales_pivot.xlsx'

# sales_pivot.to_csv(...)
# sales_pivot.to_excel(...)

